In [1]:
from IPython.display import Image, display

from react_agent.src.agents.react_agent import ReActAgent

from react_agent.src.agent_tools.sap_help_searcher import SapHelpSearcher
from react_agent.src.agent_tools.troubleshooting_searcher import TroubleshootingSearcher

from react_agent.src.util.llm_proxy import LLMProxy
from react_agent.src.util.long_term_mem_manager import PostgresMemoryManager, InMemoryManager

from react_agent.src.scripts import load_troubleshooting_memory, load_troubleshooting_postgres

In [2]:
USE_IN_MEMORY_STORE = False
WITH_ML_FLOW = False

In [3]:
# Load the initial troubleshooting memories to the long term memory store
if USE_IN_MEMORY_STORE:
    longterm_mem_manager = InMemoryManager()
    load_troubleshooting_memory.load_memories(longterm_mem_manager)
else:
    longterm_mem_manager = PostgresMemoryManager()
    load_troubleshooting_postgres.load_memories(longterm_mem_manager)

In [4]:
if(WITH_ML_FLOW):
    import mlflow
    # poetry run mlflow server --host 127.0.0.1 --port 8080
    mlflow.set_tracking_uri(uri="http://localhost:8080")
    mlflow.set_experiment("Testing of email ReAct Agent")

    mlflow.langchain.autolog()

In [5]:
tool_list = []

tool_list.append(TroubleshootingSearcher())
tool_list.append(SapHelpSearcher())

In [6]:
agent = ReActAgent(tool_list=tool_list)
#print(agent.create_sys_prompt())
#display(Image(agent.get_agent_graph()))

In [7]:
mapping_question = "Explain how 'Payment Terms' is mapped. Start with 'map_invoice1'."
extensibility_question = "As a Public Cloud customer in Spain, can I extend an existing eDocument customer invoice Process?"
peppol_support = """We as Syntax are currently implementing SAP DRC and are following the official SAP help.
We are currently at the step “Integration of Peppol Exchange with SAP S/4HANA Cloud” and there at the creation of an S/4HANA Public Cloud communication user.
A client certificate should be uploaded for the communication user to be created, as Basic Auth (via user + password) is not supported.

The problem: We do not have this certificate. Where can I get this certificate? What format must it be in?
We are stuck at this step because we are missing the certificate mentioned and it is also not clear to us what kind of certificate is meant here and where to get it.
"""
agent.run_and_print_agent_stream(user_message=peppol_support)

================================ Human Message =================================

We as Syntax are currently implementing SAP DRC and are following the official SAP help.
We are currently at the step “Integration of Peppol Exchange with SAP S/4HANA Cloud” and there at the creation of an S/4HANA Public Cloud communication user.
A client certificate should be uploaded for the communication user to be created, as Basic Auth (via user + password) is not supported.

The problem: We do not have this certificate. Where can I get this certificate? What format must it be in?
We are stuck at this step because we are missing the certificate mentioned and it is also not clear to us what kind of certificate is meant here and where to get it.

================================== Ai Message ==================================

Observation: The primary task is to identify where to obtain the client certificate required for the integration of Peppol Exchange with SAP S/4HANA Cloud, and to understand the 

In [8]:
LLMProxy().print_usage()

Call Count: 1
Token Usage: {'input_tokens': 1881, 'output_tokens': 302, 'total_tokens': 2183}
